# Joke Generator + Time Travel Explainer

This notebook shows a simple **joke generator** (topic → write joke) and then focuses on **LangGraph time travel**: how to go back to a previous checkpoint *after* the joke is generated, inspect or change state, and resume to get alternative outcomes.

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from typing import TypedDict, NotRequired
from dotenv import load_dotenv

load_dotenv()

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

## 1. State and nodes

Simple state: `topic` (from LLM) and `joke` (written from that topic).

In [ ]:
class JokeState(TypedDict):
    topic: NotRequired[str]
    joke: NotRequired[str]

In [ ]:
def generate_topic(state: JokeState):
    """LLM picks a funny topic for the joke."""
    messages = [
        SystemMessage(content="You suggest short, funny topics for jokes. Reply with only the topic, no explanation."),
        HumanMessage(content="Give me one funny topic for a joke.")
    ]
    topic = llm.invoke(messages).content
    return {"topic": topic}

def write_joke(state: JokeState):
    """LLM writes a short joke based on the topic."""
    messages = [
        SystemMessage(content="You write short, clean jokes. No setup-punchline clichés unless they're good."),
        HumanMessage(content=f"Write a short joke about: {state['topic']}")
    ]
    joke = llm.invoke(messages).content
    return {"joke": joke}

In [ ]:
workflow = StateGraph(JokeState)
workflow.add_node("generate_topic", generate_topic)
workflow.add_node("write_joke", write_joke)
workflow.add_edge(START, "generate_topic")
workflow.add_edge("generate_topic", "write_joke")
workflow.add_edge("write_joke", END)

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)
graph

## 2. Run the graph (joke generation)

We **must** pass a `thread_id` in config so the checkpointer can save every step. That’s what makes time travel possible later.

In [ ]:
config = {"configurable": {"thread_id": "joke-demo-1"}}
final_state = graph.invoke({}, config)

print("Topic:", final_state["topic"])
print()
print("Joke:", final_state["joke"])

---
## 3. Time travel (focus: *after* joke generation)

**What is time travel here?**  
LangGraph saves a **checkpoint** at every super-step. Each checkpoint is a snapshot of the graph state and "what runs next." **Time travel** means:

1. **Inspect the past** – List checkpoints for this run and see state at any step.
2. **Resume from a checkpoint** – Run again from that point (same or updated state), creating a **new fork** in history.

So *after* the joke is generated we can:
- Look at the **history** of checkpoints.
- Pick a checkpoint (e.g. *after* `generate_topic` but *before* `write_joke`).
- Optionally **update state** at that checkpoint (e.g. change `topic`).
- **Resume** from that checkpoint → only steps *after* it run again (e.g. `write_joke` runs with the new topic).

That’s the part we demo below.

### 3.1 List checkpoints (state history)

`get_state_history(config)` returns all checkpoints for this `thread_id`, **newest first**.

In [ ]:
history = list(graph.get_state_history(config))
print(f"Total checkpoints: {len(history)}")
for i, snap in enumerate(history):
    next_nodes = snap.next
    cid = snap.config["configurable"].get("checkpoint_id", "N/A")[:8]
    print(f"  {i}: next={next_nodes}  checkpoint_id=...{cid}")
    if snap.values:
        keys = list(snap.values.keys())
        print(f"      state keys: {keys}")

### 3.2 Pick a checkpoint: *after* topic, *before* joke

We want the snapshot where `generate_topic` has run (we have `topic`) but `write_joke` hasn’t. In the list above, that’s the one whose `next` is `('write_joke',)`.

In [ ]:
selected = None
for snap in history:
    if snap.next == ("write_joke",):
        selected = snap
        break

if selected is None:
    selected = history[1]  # fallback: second (older) checkpoint

print("Selected checkpoint: next =", selected.next)
print("State at this point:", selected.values)

### 3.3 Update state and resume (fork)

We **change** the topic at this checkpoint with `update_state`. That creates a **new** checkpoint (new `checkpoint_id`). Then we **resume** with `invoke(None, new_config)` so the graph runs *from that point onward* — i.e. only `write_joke` runs again, with the new topic. Result: a **different joke** from the same "time" in the graph.

In [ ]:
new_topic = "programmers and coffee"
new_config = graph.update_state(selected.config, values={"topic": new_topic})

forked_state = graph.invoke(None, new_config)

print("Forked run (after time travel + topic change):")
print("Topic:", forked_state["topic"])
print()
print("Joke:", forked_state["joke"])

### 3.4 Resume without changing state (replay)

You can also **replay** from a checkpoint without updating state: pass a config that includes that checkpoint’s `checkpoint_id` and call `invoke(None, config)`. The graph will re-execute from that point (e.g. `write_joke` runs again with the *same* topic — may give a different joke due to LLM non-determinism).

In [ ]:
replay_config = {
    "configurable": {
        "thread_id": config["configurable"]["thread_id"],
        "checkpoint_id": selected.config["configurable"]["checkpoint_id"]
    }
}
replayed_state = graph.invoke(None, replay_config)

print("Replayed run (same topic, from checkpoint):")
print("Topic:", replayed_state["topic"])
print()
print("Joke:", replayed_state["joke"])

---
## Summary

| Step | What we did |
|------|-------------|
| 1 | Ran the joke graph with a `thread_id` so every step is checkpointed. |
| 2 | Used **`get_state_history(config)`** to list all checkpoints (newest first). |
| 3 | Chose a checkpoint **after** `generate_topic` and **before** `write_joke`. |
| 4 | **Time travel + fork:** `update_state(selected.config, values={...})` then `invoke(None, new_config)` to get a new joke with a different topic. |
| 5 | **Replay:** `invoke(None, config_with_checkpoint_id)` to re-run from that point without changing state. |

So **time travel** here = go back to a saved checkpoint (after joke generation has already run once), optionally edit state, and resume to create an alternative branch. All of this is built on **persistence** (checkpointer + `thread_id`).